# Title
[]()

In [2]:
import requests
import json
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
from silvhua import *
import pickle
import os

In [3]:
with open("credentials.json") as f:
    access_token = json.load(f)['access_token']
user_id = os.environ['fb_user_id']

In [23]:
def get_user_post(user_id, access_token, pages=5, filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    user_id = str(user_id)
    url_root = "https://graph.facebook.com/v15.0/"
    url = f'{url_root}{user_id}/posts?access_token={access_token}'
    response_json_dict = dict()
    df_list = []
    for page in range(1,pages+1):
        response = requests.get(url)
        print('Response status code: ',response.status_code)
        response_json_dict[page] = response.json()
        df_list.append(json_normalize(response_json_dict[page], record_path='data'))
        try:
            url = response_json_dict[page]['paging']['next']
        except: 
            pass
    df = pd.concat(df_list)
    if filename:
        save_csv(df,filename,csv_path)
        savepickle(response_json_dict,filename,'sav',json_path)
    return df, response_json_dict

df_5p, response_json_5p = get_user_post(user_id, access_token)
df_5p

Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200


,created_time,message,id
0,2023-01-11T03:17:40+0000,There was a time when my only exercise was run...,10104327314119821_10104327972240941
1,2023-01-09T21:30:18+0000,"For the US, ""Wearable technology (#1), strengt...",10104327314119821_10104326999794731
2,2023-01-09T17:06:33+0000,Excited to see developments in wearable tech!\...,10104327314119821_10104326849426071
3,2023-01-07T21:36:58+0000,Pomelos are one of my favourite fruits. It is ...,10104327314119821_10104325303319481
4,2023-01-06T19:39:51+0000,What's your favourite app or wearable that hel...,10104327314119821_10104324466536401
...,...,...,...
18,2021-12-14T19:36:06+0000,New toy,10104327314119821_10103955698231041
19,2021-12-05T06:57:42+0000,NaN,10104327314119821_10103949690400781
20,2021-12-05T06:57:26+0000,💛,10104327314119821_10103949690325931
21,2021-12-03T22:47:08+0000,Did you know that the only way to get better i...,10104327314119821_10103948774531191


In [24]:
json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw'
csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'
filename='my_fb_posts_5page_2023-01-12'
save_csv(df_5p,filename,csv_path)
savepickle(response_json_5p,filename,'sav',json_path)

File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/my_fb_posts_5page_2023-01-12.csv
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/my_fb_posts_5page_2023-01-12.sav


In [21]:
csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'
df_5p = load_csv('my_fb_posts_5page_2023-01-12.csv', csv_path, column1_as_index=True)

Dataframe shape:  (119, 3)


In [6]:
df_5p.dtypes

Unnamed: 0       int64
created_time    object
message         object
id              object
dtype: object

In [39]:
import re
def process_post_df(df):
    """
    Convert dates in the json-derived dataframe into different formats.
    """
    regex_date = r'.+T'
    df2 = pd.DataFrame()
    df2['timestamp'] = pd.to_datetime(df['created_time'])
    df2['text'] = df['message']
    df2['date'] = df2['timestamp'].dt.date
    df2['month'] = df2['timestamp'].dt.month
    df2['day_of_week'] = df2['timestamp'].dt.dayofweek
    df2['time'] = df2['timestamp'].dt.time
    return df2

process_post_df(df_5p.head(3).copy())

,timestamp,text,date,month,day_of_week,time
0,2023-01-11 03:17:40+00:00,There was a time when my only exercise was run...,2023-01-11,1,2,03:17:40
1,2023-01-09 21:30:18+00:00,"For the US, ""Wearable technology (#1), strengt...",2023-01-09,1,0,21:30:18
2,2023-01-09 17:06:33+00:00,Excited to see developments in wearable tech!\...,2023-01-09,1,0,17:06:33


In [40]:
process_post_df(df_5p.head(3).copy()).dtypes

timestamp      datetime64[ns, UTC]
text                        object
date                        object
month                        int64
day_of_week                  int64
time                        object
dtype: object

# *End of Page*